In [ ]:
!pip install simplet5


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install rouge

In [ ]:
import os
dir_path = "/content/drive/MyDrive/To Share /dialog-bAbI-tasks/csv files"
files = os.listdir(dir_path)

In [ ]:
import os
dir_path = "/content/drive/MyDrive/To Share /full/csv files"
files = os.listdir(dir_path)

In [ ]:
for i in range(len(files)):
  print(i,files[i])

0 personalized-dialog-task1-API-calls-dev..csv
1 personalized-dialog-task1-API-calls-trn..csv
2 personalized-dialog-task1-API-calls-tst-OOV..csv
3 personalized-dialog-task1-API-calls-tst..csv
4 personalized-dialog-task2-API-refine-dev..csv
5 personalized-dialog-task2-API-refine-trn..csv
6 personalized-dialog-task2-API-refine-tst-OOV..csv
7 personalized-dialog-task2-API-refine-tst..csv
8 personalized-dialog-task1-API-calls-trn..gsheet


In [ ]:
fl_name = files[1]
fl_path = os.path.join(dir_path,fl_name)

In [ ]:
dev_path = os.path.join(dir_path,files[0])
trn_path = os.path.join(dir_path,files[1])
tst_OOV_path = os.path.join(dir_path,files[2])
tst_path = os.path.join(dir_path,files[3])


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv(fl_path)
df.head()

,Call,Response
0,<profile> female elderly <query> hi,good day madam how could i assist you today
1,<profile> female elderly <context> <user> hi <...,thank you madam i shall start the reservation now
2,<profile> female elderly <context> <user> hi <...,could you tell me your preference on the type ...
3,<profile> female elderly <context> <user> hi <...,would you mind telling me how many guests shal...
4,<profile> female elderly <context> <user> hi <...,would you mind telling me your price range


In [ ]:
train_df = pd.read_csv(trn_path)
eval_df = pd.read_csv(dev_path)
test_df = pd.read_csv(tst_path)
test_OOV_df = pd.read_csv(tst_OOV_path)
train_df.shape, eval_df.shape ,test_df.shape, test_OOV_df.shape


((36136, 2), (36086, 2), (35611, 2), (36113, 2))

In [ ]:
train_df = train_df.rename(columns={"Response":"target_text", "Call":"source_text"})
train_df = train_df[['source_text', 'target_text']]

In [ ]:
eval_df = eval_df.rename(columns={"Response":"target_text", "Call":"source_text"})
eval_df = eval_df[['source_text', 'target_text']]

In [ ]:
test_df = test_df.rename(columns={"Response":"target_text", "Call":"source_text"})
test_df = test_df[['source_text', 'target_text']]

In [ ]:
test_OOV_df = test_OOV_df.rename(columns={"Response":"target_text", "Call":"source_text"})
test_df = test_df[['source_text', 'target_text']]

In [ ]:
x_max = 0
ls_df = [train_df, eval_df,test_df, test_OOV_df]
for j in ls_df:
  df_temp = j.copy()
  for i in range(len(df_temp)):
    x = len(df_temp.loc[i]['source_text'].split())
    if x>x_max:
      x_max = x   
print(x_max)

116


In [ ]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="t5", model_name="t5-base")
model.train(train_df=train_df,
            eval_df=eval_df, 
            source_max_token_len=128, 
            target_max_token_len=50, 
            batch_size=8, max_epochs=3, use_gpu=True)

Global seed set to 42


Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

...

Global seed set to 42


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [ ]:
model.load_model("/content/outputs/simplet5-epoch-2-train-loss-0.0163-val-loss-0.0173", use_gpu=True)


In [ ]:
pred = []
for i in range(len(test_df)):

  
  pred_txt = model.predict(test_df.loc[i]['source_text'])
  pred.append(pred_txt[0])

test_df['pred_text'] = pred
test_df.to_csv('Result_task1_profile_tst.csv')


In [ ]:
pred_OOV = []
for i in range(len(test_OOV_df)):
  pred_txt_OOV = model.predict(test_OOV_df.loc[i]['source_text'])
  pred_OOV.append(pred_txt_OOV[0])

test_OOV_df['pred_text'] = pred_OOV
test_OOV_df.to_csv('Result_task1_profile_tst_OOV.csv')

## Rouge

#### For Full DF

In [ ]:
pip install rouge
from rouge import Rouge
rouge = Rouge()

model_out = test_df['pred_text'].to_list()
reference = test_df['target_text'].to_list()

rouge.get_scores(model_out, reference,avg=True)

{'rouge-1': {'f': 0.8807924892595469,
  'p': 0.8810958406541431,
  'r': 0.8805660674865979},
 'rouge-2': {'f': 0.8781179901486494,
  'p': 0.8783816506735148,
  'r': 0.8779219883317566},
 'rouge-l': {'f': 0.8807767213629842,
  'p': 0.8810761307834399,
  'r': 0.8805529275727958}}

## For API Call

In [ ]:
df = test_OOV_df.copy()

ls_api_pred = []
ls_api_targ = []
c = 0
for i in range(len(df)):
    #if c >100: break
    if 'api_call' in str(df.loc[i][2]):
        ls_api_pred.append(df.loc[i][3].split()[1:])
        ls_api_targ.append(df.loc[i][2].split()[1:])
        c+=1
print(c)
len(ls_api_targ)
        

#### BLEU-4

In [ ]:
ls_api_scores = []
for i in range(len(ls_api_targ)):
    reference = [ls_api_targ[i]]
    candidate = ls_api_pred[i]
    score = sentence_bleu(reference, candidate, weights=(0, 0, 0, 1))
    ls_api_scores.append(round(score,2)) 
len(ls_api_scores)
Counter(ls_api_scores)

#### Rouge Score

In [ ]:
from rouge import Rouge
rouge = Rouge()

model_out = ls_api_pred
reference = ls_api_targ

rouge.get_scores(model_out, reference,avg=True)